In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import get_raw_data
import get_preprocessing_data
import analysis_method
import statsmodels.regression.linear_model as sm

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

pv_list = ["0102", "1701", "1702", "0111", "0330", "0331", "0332", "0333", "0101", "0107", "0110", "5501"]
#["0102"(가구주와의 관계), "1701"(근로소득 유무), "1702"(총 연간근로소득), "0111"(학력졸업여부), "0330"(업종8차),
# "0331"(업종9차), "0332"(직종8차), "0333"(직종9차), "0101"(성별), "0107"(만나이), "0110"(학력학교), "5501"(혼인상태)]
hv_list = ["0150", "1406", "2102", "2134", "2136", " 2138", "2140", "2142", "2152", "2153", "2155",
           "2156", "2157", "2158", "2159", "2160", "2182", "2183", "2184", "2185", "2186", "2187",
           "2189", "2190", "2191"]
income_code_list = ["2102", "2134", "2136", " 2138", "2140", "2142", "2152", "2153", "2155", "2156",
                    "2157", "2158", "2159", "2160", "2182", "2183", "2184", "2185", "2186", "2187",
                    "2189", "2190", "2191"]

cpi_data = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\CPI.xls").loc[0, :]
cpi_data_index = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\CPI.xls", dtype="str").loc[14, :].values[1:]
cpi_data = pd.Series(cpi_data.values[1:], dtype="float")
cpi_data.name = "CPI"
cpi_data.index = cpi_data_index

industry_8 = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\국내데이터_산업분류.xlsx", sheet_name="8차")
industry_large = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\국내데이터_산업분류.xlsx", sheet_name="대분류")

kospi_return = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\kospi_return.xlsx", index_col=0)

data_num = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21"]

cpi_multiple = get_preprocessing_data.get_cpi_multi(cpi_data, data_num)

data_dict = {}
for i in data_num:
    h_path = "C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\h_data\\h_" + i + ".pickle"
    p_path = "C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\p_data\\p_" + i + ".pickle"

    h_data = pd.read_pickle(h_path)
    p_data = pd.read_pickle(p_path)

    raw_hdata = get_raw_data.get_hv_data(h_data, hv_list, i)
    raw_pdata = get_raw_data.get_pv_data(p_data, pv_list, i)
    
    fin_p_data = get_preprocessing_data.get_fin_personal(raw_pdata, industry_8, industry_large)
    income_data = get_preprocessing_data.get_income_data(fin_p_data.index, raw_hdata)

    fin_x_value = get_preprocessing_data.get_final_xvalue(raw_hdata, fin_p_data)
    fin_y_value = get_preprocessing_data.get_final_yvalue(income_data, cpi_multiple[i])

    finally_preprocessed_data = get_preprocessing_data.get_total_data(fin_y_value, fin_x_value)

    data_dict[i] = finally_preprocessed_data

C:\Users\JeayongYu\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data_df = pd.DataFrame()
for i in data_dict.keys():
    data_df = pd.concat([data_df, data_dict[i]])

_______________________________


### log(Y*)의 계산

In [3]:
temp_edu = data_df.groupby(["0110", "0330"]).get_group((1, "제조업 (10 ~ 33)"))
temp_edu = get_preprocessing_data.get_agecut_data(temp_edu, 22)
x_mat = analysis_method.get_reg_x(temp_edu).iloc[:, :3]
fer_results = analysis_method.fe_regression_sol(temp_edu, False)

poly_x_mat = analysis_method.polynomial_matrix(temp_edu["0107"], 3) .iloc[:, 1:]
poly_x_mat.index = x_mat.index
fitting_result = analysis_method.fitting_polynomial(fer_results, 3, False)

In [4]:
fer_results[0][:3]

intercept     7.671403
marriage      0.216859
family_num    0.068147
dtype: float64

In [5]:
fitting_result[1]

intercept    -0.310767
age^1/10^0   -0.027853
age^2/10^1    0.015535
age^3/10^2   -0.001434
dtype: float64

In [6]:
fitting_paramters = pd.concat([fer_results[0][:3], fitting_result[1]], axis=1).sum(axis=1)
fitting_paramters

intercept     7.360636
marriage      0.216859
family_num    0.068147
age^1/10^0   -0.027853
age^2/10^1    0.015535
age^3/10^2   -0.001434
dtype: float64

In [7]:
real_data = pd.concat([x_mat, poly_x_mat], axis=1)
real_data

,intercept,marriage,family_num,age^1/10^0,age^2/10^1,age^3/10^2
hhid,,,,,,
91.0,1,1.0,5.0,51.0,260.1,1326.51
99.0,1,1.0,4.0,51.0,260.1,1326.51
115.0,1,1.0,4.0,43.0,184.9,795.07
173.0,1,1.0,4.0,38.0,144.4,548.72
175.0,1,1.0,4.0,42.0,176.4,740.88
...,...,...,...,...,...,...
110251.0,1,0.0,1.0,24.0,57.6,138.24
101398.0,1,1.0,3.0,61.0,372.1,2269.81
110252.0,1,0.0,1.0,24.0,57.6,138.24


In [8]:
temp_edu["labor income"]

hhid
91.0        1940.968781
99.0        3105.550050
115.0       4852.421952
173.0       3881.937562
175.0       3881.937562
               ...     
110251.0    2410.000000
101398.0    4360.000000
110252.0    2760.000000
110085.0    6140.000000
110102.0    3500.000000
Name: labor income, Length: 6004, dtype: float64

In [9]:
real_data @ fitting_paramters

hhid
91.0        8.636426
99.0        8.568280
115.0       8.384863
173.0       8.248181
175.0       8.358367
              ...   
110251.0    7.456930
101398.0    8.608988
110252.0    7.456930
110085.0    8.163537
110102.0    7.889237
Length: 6004, dtype: float64

_______________________________


### 3rd-order polynomials vision(각 parameter의 계수를 한번 추정하여 rid계산하는 경우)

In [10]:
fitting_paramters

intercept     7.360636
marriage      0.216859
family_num    0.068147
age^1/10^0   -0.027853
age^2/10^1    0.015535
age^3/10^2   -0.001434
dtype: float64

----------------------------------------
### shock calculation

In [11]:
industry_list = data_df["0330"].value_counts()[:15].index
industry_list

Index(['제조업 (10 ~ 33)', '도매 및 소매업 (45~47)', '건설업 (41 ~ 42)', '운수업(49~52)',
       '교육 서비스업(85)', '협회 및 단체, 수리 및 기타 개인 서비스 업(94~96)',
       '공공행정, 국방 및 사회보장 행정(84)', '출판, 영상, 방송통 신 및 정보서비스업 (58 ~ 63)',
       '금융 및 보험업 (64 ~ 66)', '농업, 임업 및 어업 (01 ~ 03)', '부동산업 및 임대업 (68 ~ 69)',
       '숙박 및 음식점업 (55 ~ 56)', '하수 · 폐기물 처리, 원료재생 및 환경복 원업 (37 ~ 39)',
       '전문, 과학 및 기술 서비스업 (70 ~ 73)', '보건업 및 사회복지 서비스업(86~87)'],
      dtype='object')

In [12]:
group_sigma_dict = {} 
group_rid_dict = {}
for l in industry_list:
    group_sigma_df = pd.DataFrame()
    group_pvalue_df = pd.DataFrame()
    group_rid_df = pd.DataFrame()

    start_list = [22, 24, 26]
    for i in range(data_df["0110"].max()):
        df_y_star = pd.DataFrame()

        temp_edu = data_df.groupby(["0110", "0330"]).get_group((i+1, l))
        temp_edu = get_preprocessing_data.get_agecut_data(temp_edu, start_list[i])

        fer_results = analysis_method.fe_regression_sol(temp_edu, False)
        fitting_result = analysis_method.fitting_polynomial(fer_results, 3, False)

        fitting_paramters = pd.concat([fer_results[0][:3], fitting_result[1]], axis=1).sum(axis=1)

        for j in data_num:
            temp_edu_split = data_dict[j].groupby(["0110", "0330"]).get_group((i+1, l))
            temp_edu_split = get_preprocessing_data.get_agecut_data(temp_edu_split, start_list[i])

            log_y = np.log(temp_edu_split["labor income"])

            x_mat = analysis_method.get_reg_x(temp_edu_split).iloc[:, :3]
            poly_x_mat = analysis_method.polynomial_matrix(temp_edu_split["0107"], 3) .iloc[:, 1:]
            poly_x_mat.index = x_mat.index

            temp_real_data = pd.concat([x_mat, poly_x_mat], axis=1)

            f_hat = temp_real_data @ fitting_paramters

            log_y_star = log_y - f_hat
            df_y_star = pd.concat([df_y_star, log_y_star], axis=1)

        df_y_star.columns = data_num

        temp_varrid_list = []
        temp_rid_list = []
        for k in range(len(data_num)-1):
            temp_rid = df_y_star.iloc[:, :-(k+1)].values - df_y_star.iloc[:, k+1:].values
            temp_rid_list.append(temp_rid)

            temp_varrid_list.append(np.nanvar(temp_rid))

        var_rid_series = pd.Series(temp_varrid_list)

        rid_series = (pd.DataFrame(temp_rid_list[0], columns = data_num[1:]) * -1).mean()
        group_rid_df = pd.concat([group_rid_df, rid_series], axis=1)

        var_rid_x = np.ones([len(var_rid_series), 2]) * 2
        var_rid_x[:, 0] = var_rid_series.index.values + 1
        var_rid_y = var_rid_series.values

        sol = sm.OLS(var_rid_y, var_rid_x)
        sol_result = sol.fit()

        sigma_df = pd.DataFrame(sol_result.params, index=["permanent_shock", "idiosyncratic_temporary_shock"], columns=["sigma"])
        pvalue_df = pd.DataFrame(np.round(sol_result.pvalues, 3), index=["permanent_shock(p-value)", "idiosyncratic_temporary_shock(p-value)"], columns=["p-value"])

        group_sigma_df = pd.concat([group_sigma_df, sigma_df], axis=1)
        group_pvalue_df = pd.concat([group_pvalue_df, pvalue_df], axis=1)

    group_rid_df.columns = ["high_school", "college", "university"]
    group_sigma_df.columns = ["high_school", "college", "university"]
    group_pvalue_df.columns = ["high_school", "college", "university"]
    group_sigma_df = pd.concat([group_sigma_df, group_pvalue_df]).loc[["idiosyncratic_temporary_shock", "permanent_shock", "idiosyncratic_temporary_shock(p-value)", "permanent_shock(p-value)"],:]
    group_sigma_dict[l] = group_sigma_df
    group_rid_dict[l] = group_rid_df

C:\Users\JeayongYu\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: RuntimeWarning: Degrees of freedom <= 0 for slice.
C:\Users\JeayongYu\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\JeayongYu\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\JeayongYu\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [13]:
pd.DataFrame.from_dict({(i,j): group_sigma_dict[i][j]
                        for i in group_sigma_dict.keys()
                        for j in group_sigma_dict[i].columns}, orient="index")

idiosyncratic_temporary_shock  \
제조업 (10 ~ 33)                        high_school                       0.128567   
                                     college                           0.132352   
                                     university                        0.098062   
도매 및 소매업 (45~47)                     high_school                       0.230334   
                                     college                           0.152413   
                                     university                        0.180441   
건설업 (41 ~ 42)                        high_school                       0.161094   
                                     college                           0.026122   
                                     university                        0.105717   
운수업(49~52)                           high_school                       0.110736   
                                     college                           0.104665   
                                     university                        0.102564   
교육 서비스업(85)                          high_school                       0.165170   
                                     college                                NaN   
                                     university                        0.097606   
협회 및 단체, 수리 및 기타 개인 서비스 업(94~96)     high_school                       0.205737   
                                     college                           0.097313   
                                     university                        0.210729   
공공행정, 국방 및 사회보장 행정(84)               high_school                       0.063387   
                                     college                           0.080998   
                                     university                        0.130281   
출판, 영상, 방송통 신 및 정보서비스업 (58 ~ 63)     high_school                       0.176051   
                                     college                           0.232318   
                                     university                        0.156717   
금융 및 보험업 (64 ~ 66)                   high_school                       0.291905   
                                     college                           0.047781   
                                     university                        0.088360   
농업, 임업 및 어업 (01 ~ 03)                high_school                       0.326568   
                                     college                           0.719001   
                                     university                        0.286442   
부동산업 및 임대업 (68 ~ 69)                 high_school                       0.319312   
                                     college                           0.142916   
                                     university                             NaN   
숙박 및 음식점업 (55 ~ 56)                  high_school                       0.270627   
                                     college                           0.275648   
                                     university                        0.417059   
하수 · 폐기물 처리, 원료재생 및 환경복 원업 (37 ~ 39) high_school                       0.148473   
                                     college                                NaN   
                                     university                        0.149371   
전문, 과학 및 기술 서비스업 (70 ~ 73)           high_school                       0.334095   
                                     college                                NaN   
                                     university                        0.211715   
보건업 및 사회복지 서비스업(86~87)               high_school                            NaN   
                                     college                                NaN   
                                     university                        0.122719   

                                                  permanent_shock  \
제조업 (10 ~ 33)                        high_school         0.005564   
                                     college             0.004175   
                        

----------------------------------------
### Beta & Correlation

In [14]:
ksi = group_rid_dict
ksi = pd.DataFrame.from_dict({(i,j): ksi[i][j]
                        for i in ksi.keys()
                        for j in ksi[i].columns}, orient="columns")
ksi

제조업 (10 ~ 33)                      도매 및 소매업 (45~47)                       \
     high_school   college university      high_school   college university   
02      0.007936 -0.015213  -0.168354         0.047998 -0.216268   0.005373   
03      0.121108  0.109246   0.102576         0.037752  0.107304   0.070973   
04     -0.023950  0.056576   0.091451         0.009540 -0.170712   0.329930   
05      0.122707  0.274583   0.085298         0.170987  0.363985   0.159425   
06      0.020656 -0.052918   0.032888         0.030831  0.033840  -0.027450   
07      0.074780  0.131687   0.041172        -0.016062  0.087894  -0.042301   
08      0.083470  0.099782   0.080112         0.086254  0.050811   0.146413   
09      0.047586  0.082750   0.043073         0.120813  0.091605   0.049105   
10      0.078421  0.250875   0.031349        -0.042151  0.096209   0.155673   
11      0.059346  0.094015  -0.009247         0.067231 -0.016178   0.027143   
12     -0.058674 -0.055999  -0.036908        -0.063784 -0.008752   0.034829   
13      0.018799  0.007880   0.031020         0.134383 -0.020572   0.095717   
14      0.075154  0.014054   0.122203         0.121494  0.007894  -0.069367   
15      0.030897  0.035795   0.028640        -0.070758  0.042744   0.040489   
16      0.044527  0.037914   0.067321         0.058498 -0.028287   0.005821   
17      0.026370  0.030604   0.009034         0.037838  0.057611  -0.048287   
18      0.014769 -0.002349   0.001254         0.029775  0.057380   0.035751   
19      0.044185  0.045973  -0.005797         0.040638  0.023579  -0.024350   
20      0.006813  0.057697  -0.017589        -0.009437 -0.014199   0.081272   
21      0.028423 -0.052852   0.037628         0.043337 -0.045669   0.020934   

   건설업 (41 ~ 42)                       운수업(49~52)  ... 숙박 및 음식점업 (55 ~ 56)  \
     high_school   college university high_school  ...          university   
02     -0.010032  0.089944  -0.111470    0.040137  ...           -0.113438   
03      0.159192  0.145865  -0.030692   -0.022948  ...           -0.055891   
04     -0.040508 -0.018140   0.034383   -0.046301  ...            0.169800   
05      0.340476  0.113331   0.055930    0.190651  ...            0.013178   
06      0.081855  0.067354   0.143102    0.059257  ...            0.169455   
07      0.060136  0.063546   0.108684    0.088376  ...           -0.050348   
08     -0.017736  0.460501   0.156956    0.005948  ...           -0.076455   
09      0.152887  0.234690   0.053747    0.074090  ...            0.323796   
10      0.019184  0.107719  -0.018915    0.068813  ...            0.325086   
11      0.015109  0.063075   0.092624    0.013874  ...            0.213155   
12     -0.024735 -0.024850  -0.024349   -0.075282  ...           -0.261137   
13      0.130247 -0.000034   0.183821    0.116805  ...            0.137881   
14      0.054316 -0.051046   0.109588    0.041827  ...           -0.008953   
15      0.112956  0.036584   0.013071    0.060065  ...           -0.013589   
16      0.052958  0.006262  -0.017650    0.004752  ...            0.003322   
17      0.103464 -0.000543   0.020792    0.061517  ...            0.002578   
18      0.022046 -0.003612   0.007192    0.016380  ...           -0.106549   
19      0.069472  0.018347   0.032737    0.023663  ...           -0.091195   
20      0.050565  0.081795  -0.020756    0.140234  ...            0.284360   
21      0.010901  0.021683   0.034308    0.031035  ...           -0.009280   

   하수 · 폐기물 처리, 원료재생 및 환경복 원업 (37 ~ 39)                       \
                            high_school   college university   
02                            -0.114693 -0.205260   0.151646   
03                             0.345524  0.054593  -0.080510   
04                            -0.081719  0.176880   0.217866   
05                            -0.276414 -0.092816  -0.172874   
06                             0.042524  0.164031   0.194906   
07                             0.336040  0.420288  -0.062724   
08                            -0.00

In [15]:
kospi_return

,코스피
Symbol Name,
1,-42.21
2,49.47
3,82.78
4,-49.11
5,37.47
6,-9.54
7,29.19
8,10.51
9,55.99


In [16]:
kospi_return.index = data_num
kospi_ret = kospi_return.copy() / 100

beta_dict = {}
for j in group_rid_dict.keys():
    beta_list = []
    pvalue_list = []
    for i in group_rid_dict[j].columns:
        temp_eta = (kospi_ret - 0.02) - np.mean(kospi_ret - 0.02) 
        temp_beta_model = sm.OLS(ksi[j][i], temp_eta.loc[ksi[j].index])
        temp_beta_results = temp_beta_model.fit()
        #print("----------------------------------------------------------")
        #print(i, j)
        #print(temp_beta_results.summary())

        beta_list.append(temp_beta_results.params[0])
        pvalue_list.append(temp_beta_results.pvalues[0])

    beta = pd.DataFrame(beta_list, index=["high_school", "college", "university"], columns=["Beta"])
    pvalue = pd.DataFrame(pvalue_list, index=["high_school", "college", "university"], columns=["p-value"])
    
    beta_dict[j] = pd.concat([beta, pvalue], axis=1)

In [17]:
pd.DataFrame.from_dict({(i,j): beta_dict[i][j]
                        for i in beta_dict.keys()
                        for j in beta_dict[i].columns}, orient="index")

high_school   college  \
제조업 (10 ~ 33)                        Beta        0.101826  0.093955   
                                     p-value     0.013192  0.214464   
도매 및 소매업 (45~47)                     Beta        0.125151  0.100652   
                                     p-value     0.016377  0.229080   
건설업 (41 ~ 42)                        Beta        0.152147  0.137073   
                                     p-value     0.045933  0.150711   
운수업(49~52)                           Beta        0.082350 -0.066043   
                                     p-value     0.128514  0.391850   
교육 서비스업(85)                          Beta        0.221695  1.340621   
                                     p-value     0.004868  0.016913   
협회 및 단체, 수리 및 기타 개인 서비스 업(94~96)     Beta        0.156951 -0.038818   
                                     p-value     0.079351  0.605256   
공공행정, 국방 및 사회보장 행정(84)               Beta       -0.002346  0.058043   
                                     p-value     0.971821  0.354391   
출판, 영상, 방송통 신 및 정보서비스업 (58 ~ 63)     Beta        0.055665  0.027564   
                                     p-value     0.552955  0.792812   
금융 및 보험업 (64 ~ 66)                   Beta        0.142087  0.088151   
                                     p-value     0.312728  0.226062   
농업, 임업 및 어업 (01 ~ 03)                Beta        0.245431  0.334131   
                                     p-value     0.007991  0.292476   
부동산업 및 임대업 (68 ~ 69)                 Beta        0.047680 -0.160298   
                                     p-value     0.598997  0.245900   
숙박 및 음식점업 (55 ~ 56)                  Beta        0.046525  0.032863   
                                     p-value     0.642149  0.877610   
하수 · 폐기물 처리, 원료재생 및 환경복 원업 (37 ~ 39) Beta        0.124678 -0.128810   
                                     p-value     0.222906  0.388238   
전문, 과학 및 기술 서비스업 (70 ~ 73)           Beta       -0.051293 -0.012482   
                                     p-value     0.584333  0.936290   
보건업 및 사회복지 서비스업(86~87)               Beta        0.097127  0.019806   
                                     p-value     0.451280  0.896166   

                                              university  
제조업 (10 ~ 33)                        Beta       0.014368  
                                     p-value    0.774988  
도매 및 소매업 (45~47)                     Beta      -0.051192  
                                     p-value    0.509098  
건설업 (41 ~ 42)                        Beta       0.024158  
                                     p-value    0.694306  
운수업(49~52)                           Beta       0.083861  
                                     p-value    0.195569  
교육 서비스업(85)                          Beta       0.026588  
                                     p-value    0.610871  
협회 및 단체, 수리 및 기타 개인 서비스 업(94~96)     Beta       0.052863  
                                     p-value    0.475946  
공공행정, 국방 및 사회보장 행정(84)               Beta       0.131062  
                                     p-value    0.012289  
출판, 영상, 방송통 신 및 정보서비스업 (58 ~ 63)     Beta       0.028154  
                                     p-value    0.617077  
금융 및 보험업 (64 ~ 66)                   Beta      -0.072429  
                                     p-value    0.226886  
농업, 임업 및 어업 (01 ~ 03)                Beta       0.053484  
                                     p-value    0.679201  
부동산업 및 임대업 (68 ~ 69)                 Beta      -0.327695  
                                     p-value    0.069147  
숙박 및 음식점업 (55 ~ 56)                  Beta       0.065119  
                                     p-value    0.588822  
하수 · 폐기물 처리, 원료재생 및 환경복 원업 (37 ~ 39) Beta      -0.092536  
                                     p-value    0.223831  
전문, 과학 및 기술 서비스업 (70 ~ 73)           Beta       0.064404  
                                     p-value    0.690927  
보건업 및 사회복지 서비스업(86~87)               Beta      -0.071205  
                                     p-value    0.440590

In [18]:
corr_dict = {}
for j in group_rid_dict.keys():
    corr_list = []
    for i in [0, 1, 2]:
        corr_list.append(np.corrcoef(temp_eta.loc[ksi[j].index, :].코스피, ksi[j][ksi[j].columns[i]])[0, 1])

    corr_dict[j] = pd.DataFrame(corr_list, index=ksi[j].columns, columns=["correlation coefficent"])

In [19]:
corr_df = pd.DataFrame.from_dict({(i,j): corr_dict[i][j]
                        for i in corr_dict.keys()
                        for j in corr_dict[i].columns}, orient="index")
corr_df.index = corr_df.index.levels[0]
corr_df

,high_school,college,university
건설업 (41 ~ 42),0.652772,0.282691,0.032918
"공공행정, 국방 및 사회보장 행정(84)",0.565091,0.262454,-0.227488
교육 서비스업(85),0.494910,0.330924,0.055284
금융 및 보험업 (64 ~ 66),0.363212,-0.279609,0.301801
"농업, 임업 및 어업 (01 ~ 03)",0.595413,0.515874,0.086030
도매 및 소매업 (45~47),0.391045,-0.159060,0.141180
보건업 및 사회복지 서비스업(86~87),-0.060667,0.198695,0.583032
부동산업 및 임대업 (68 ~ 69),0.108839,0.040393,0.084000
숙박 및 음식점업 (55 ~ 56),0.215606,0.270681,-0.318865
운수업(49~52),0.575193,0.236904,0.065042


In [21]:
corr_df.to_excel("./results5.xlsx")

_______________________________


### 3rd-order polynomials vision(각 parameter의 계수를 매번 추정하여 rid계산하는 경우)

In [20]:
group_sigma_dict = {} 
group_rid_dict = {}
for l in industry_list:
    group_sigma_df = pd.DataFrame()
    group_pvalue_df = pd.DataFrame()
    group_rid_df = pd.DataFrame()

    start_list = [22, 24, 26]
    for i in range(data_df["0110"].max()):
        df_y_star = pd.DataFrame()

        for j in data_num:
            temp_edu_split = data_dict[j].groupby(["0110", "0330"]).get_group((i+1, l))
            temp_edu_split = get_preprocessing_data.get_agecut_data(temp_edu_split, start_list[i])

            fer_results = analysis_method.fe_regression_sol(temp_edu_split, False)
            fitting_result = analysis_method.fitting_polynomial(fer_results, 3, False)

            fitting_paramters = pd.concat([fer_results[0][:3], fitting_result[1]], axis=1).sum(axis=1)

            log_y = np.log(temp_edu_split["labor income"])

            x_mat = analysis_method.get_reg_x(temp_edu_split).iloc[:, :3]
            poly_x_mat = analysis_method.polynomial_matrix(temp_edu_split["0107"], 3) .iloc[:, 1:]
            poly_x_mat.index = x_mat.index

            temp_real_data = pd.concat([x_mat, poly_x_mat], axis=1)

            f_hat = temp_real_data @ fitting_paramters

            log_y_star = log_y - f_hat
            df_y_star = pd.concat([df_y_star, log_y_star], axis=1)

        df_y_star.columns = data_num

        temp_varrid_list = []
        temp_rid_list = []
        for k in range(len(data_num)-1):
            temp_rid = df_y_star.iloc[:, :-(k+1)].values - df_y_star.iloc[:, k+1:].values
            temp_rid_list.append(temp_rid)

            temp_varrid_list.append(np.nanvar(temp_rid))

        var_rid_series = pd.Series(temp_varrid_list)

        rid_series = (pd.DataFrame(temp_rid_list[0], columns = data_num[1:]) * -1).mean()
        group_rid_df = pd.concat([group_rid_df, rid_series], axis=1)

        var_rid_x = np.ones([len(var_rid_series), 2]) * 2
        var_rid_x[:, 0] = var_rid_series.index.values + 1
        var_rid_y = var_rid_series.values

        sol = sm.OLS(var_rid_y, var_rid_x)
        sol_result = sol.fit()

        sigma_df = pd.DataFrame(sol_result.params, index=["permanent_shock", "idiosyncratic_temporary_shock"], columns=["sigma"])
        pvalue_df = pd.DataFrame(np.round(sol_result.pvalues, 3), index=["permanent_shock(p-value)", "idiosyncratic_temporary_shock(p-value)"], columns=["p-value"])

        group_sigma_df = pd.concat([group_sigma_df, sigma_df], axis=1)
        group_pvalue_df = pd.concat([group_pvalue_df, pvalue_df], axis=1)

    group_rid_df.columns = ["high_school", "college", "university"]
    group_sigma_df.columns = ["high_school", "college", "university"]
    group_pvalue_df.columns = ["high_school", "college", "university"]
    group_sigma_df = pd.concat([group_sigma_df, group_pvalue_df]).loc[["idiosyncratic_temporary_shock", "permanent_shock", "idiosyncratic_temporary_shock(p-value)", "permanent_shock(p-value)"],:]
    group_sigma_dict[l] = group_sigma_df
    group_rid_dict[l] = group_rid_df

C:\Users\JeayongYu\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1578: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
C:\Users\JeayongYu\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: Degrees of freedom <= 0 for slice.
C:\Users\JeayongYu\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\JeayongYu\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\JeayongYu\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
C:\Users\JeayongYu\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1578: RuntimeWarning: invalid value encountered in double_scalars
  return np.do

In [21]:
pd.DataFrame.from_dict({(i,j): group_sigma_dict[i][j]
                        for i in group_sigma_dict.keys()
                        for j in group_sigma_dict[i].columns}, orient="index")

idiosyncratic_temporary_shock  \
제조업 (10 ~ 33)                        high_school                       0.131300   
                                     college                           0.132130   
                                     university                        0.095492   
도매 및 소매업 (45~47)                     high_school                       0.224516   
                                     college                           0.151838   
                                     university                        0.189176   
건설업 (41 ~ 42)                        high_school                       0.162356   
                                     college                           0.025824   
                                     university                        0.134736   
운수업(49~52)                           high_school                       0.107727   
                                     college                           0.201969   
                                     university                        0.127112   
교육 서비스업(85)                          high_school                       0.157626   
                                     college                                NaN   
                                     university                        0.097859   
협회 및 단체, 수리 및 기타 개인 서비스 업(94~96)     high_school                       0.205266   
                                     college                           0.194724   
                                     university                        0.268670   
공공행정, 국방 및 사회보장 행정(84)               high_school                       0.081326   
                                     college                           0.233748   
                                     university                        0.117209   
출판, 영상, 방송통 신 및 정보서비스업 (58 ~ 63)     high_school                       0.185152   
                                     college                           0.222042   
                                     university                        0.145063   
금융 및 보험업 (64 ~ 66)                   high_school                       0.644579   
                                     college                           0.216731   
                                     university                        0.103247   
농업, 임업 및 어업 (01 ~ 03)                high_school                       0.388229   
                                     college                           0.766842   
                                     university                        0.289640   
부동산업 및 임대업 (68 ~ 69)                 high_school                       0.358274   
                                     college                           0.293913   
                                     university                             NaN   
숙박 및 음식점업 (55 ~ 56)                  high_school                       0.262544   
                                     college                           0.101737   
                                     university                        0.557487   
하수 · 폐기물 처리, 원료재생 및 환경복 원업 (37 ~ 39) high_school                       0.159275   
                                     college                                NaN   
                                     university                        0.128983   
전문, 과학 및 기술 서비스업 (70 ~ 73)           high_school                       0.670104   
                                     college                                NaN   
                                     university                        0.156812   
보건업 및 사회복지 서비스업(86~87)               high_school                            NaN   
                                     college                                NaN   
                                     university                        0.297500   

                                                  permanent_shock  \
제조업 (10 ~ 33)                        high_school         0.003629   
                                     college             0.002630   
                        

----------------------------------------
### Beta & Correlation

In [22]:
ksi = group_rid_dict
ksi = pd.DataFrame.from_dict({(i,j): ksi[i][j]
                        for i in ksi.keys()
                        for j in ksi[i].columns}, orient="columns")
ksi

제조업 (10 ~ 33)                      도매 및 소매업 (45~47)                       \
     high_school   college university      high_school   college university   
02      0.092415 -0.009550  -0.108582        -0.048245 -0.026482   0.151941   
03     -0.046373  0.217604  -0.003327         0.050474  0.046641   0.041362   
04      0.019200 -0.019377   0.078028         0.026100 -0.101876   0.007340   
05     -0.024030 -0.029735  -0.041051         0.006763 -0.000850   0.076240   
06      0.017587  0.072216   0.114929         0.017227  0.115254   0.014672   
07      0.032758  0.004603  -0.068730        -0.049780  0.064153  -0.010763   
08      0.047277  0.041259  -0.003733         0.021206  0.045778   0.055781   
09      0.063057  0.077727  -0.011944         0.036601 -0.054099   0.068683   
10      0.039875  0.119233   0.066719         0.092201  0.066329   0.080652   
11      0.032267  0.013761   0.051982        -0.008510 -0.028098  -0.038891   
12      0.011383 -0.005527  -0.054034         0.046088  0.059454   0.066194   
13     -0.007781 -0.019893   0.079352         0.076391 -0.015814   0.044211   
14      0.023986  0.091285   0.025838         0.028860 -0.051088  -0.059886   
15      0.036390 -0.018748   0.079790        -0.026655 -0.052164   0.093505   
16      0.034371  0.048165  -0.030382         0.001275  0.038669  -0.039232   
17      0.013681 -0.065387   0.046244         0.013543  0.021380   0.088856   
18      0.002825  0.009305  -0.043872        -0.006112  0.023294  -0.050056   
19     -0.003027  0.040019   0.017304         0.038317  0.017456  -0.016980   
20      0.002110  0.005271  -0.001631         0.034429 -0.116501  -0.024418   
21      0.028775 -0.050063  -0.031348        -0.010141  0.064937   0.103197   

   건설업 (41 ~ 42)                       운수업(49~52)  ... 숙박 및 음식점업 (55 ~ 56)  \
     high_school   college university high_school  ...          university   
02      0.152999  0.073505  -0.030450   -0.051397  ...           -0.269503   
03     -0.032875  0.142461   0.075912   -0.015789  ...           -0.109219   
04      0.179117  0.050606  -0.212111    0.010137  ...           -0.666608   
05      0.002884  0.157720   0.127237    0.014180  ...            0.070189   
06      0.097840  0.030028  -0.044082    0.102578  ...            0.076104   
07      0.011923  0.462180   0.070665   -0.001645  ...           -0.073460   
08      0.086747  0.024531   0.137769    0.069378  ...            0.094827   
09      0.010543  0.223696   0.006391    0.014384  ...            0.002409   
10      0.016616  0.042938   0.061254    0.031371  ...            0.315000   
11     -0.012474 -0.019380   0.019831    0.036969  ...           -0.271268   
12      0.082021  0.164682   0.190042    0.004265  ...           -0.148580   
13      0.058804  0.021186  -0.006789    0.084902  ...           -0.000428   
14      0.090206 -0.097839   0.059896    0.046227  ...            0.075869   
15      0.018351 -0.033307  -0.054787    0.012588  ...           -0.020242   
16      0.045054  0.012334   0.133604    0.031198  ...            0.048494   
17      0.022169 -0.029435   0.069085    0.033919  ...           -0.220776   
18      0.040155 -0.070942  -0.026735   -0.008029  ...            0.111482   
19      0.001185  0.011880  -0.001607    0.043517  ...           -0.034310   
20      0.032061  0.025728  -0.021070    0.002541  ...            0.110566   
21      0.000150  0.030181  -0.002037   -0.004846  ...           -0.045260   

   하수 · 폐기물 처리, 원료재생 및 환경복 원업 (37 ~ 39)                           \
                            high_school       college university   
02                            -0.067312  3.964277e-01  -0.006549   
03                            -0.019372 -3.895362e-01   0.183080   
04                            -0.024367  6.957398e-14   0.135122   
05                            -0.122792  2.168141e-01  -0.139505   
06                            -0.143433  1.393805e-01  -0.113716   
07                            -0.046284 -1.010969e-11   0.254403   
08 

In [23]:
ksi = group_rid_dict

kospi_return.index = data_num
kospi_ret = kospi_return.copy() / 100


beta_dict = {}
for j in group_rid_dict.keys():
    beta_list = []
    pvalue_list = []
    for i in group_rid_dict[j].columns:
        temp_eta = (kospi_ret - 0.02) - np.mean(kospi_ret - 0.02) 
        temp_beta_model = sm.OLS(group_rid_dict[j][i], temp_eta.loc[group_rid_dict[j].index])
        temp_beta_results = temp_beta_model.fit()
        #print("----------------------------------------------------------")
        #print(i, j)
        #print(temp_beta_results.summary())

        beta_list.append(temp_beta_results.params[0])
        pvalue_list.append(temp_beta_results.pvalues[0])

    beta = pd.DataFrame(beta_list, index=["high_school", "college", "university"], columns=["Beta"])
    pvalue = pd.DataFrame(pvalue_list, index=["high_school", "college", "university"], columns=["p-value"])
    
    beta_dict[j] = pd.concat([beta, pvalue], axis=1)

In [24]:
pd.DataFrame.from_dict({(i,j): beta_dict[i][j]
                        for i in beta_dict.keys()
                        for j in beta_dict[i].columns}, orient="index")

high_school   college  \
제조업 (10 ~ 33)                        Beta        0.000643  0.088068   
                                     p-value     0.980979  0.075378   
도매 및 소매업 (45~47)                     Beta        0.001682  0.006992   
                                     p-value     0.954461  0.874527   
건설업 (41 ~ 42)                        Beta       -0.064237  0.120215   
                                     p-value     0.209156  0.224438   
운수업(49~52)                           Beta       -0.011393 -0.143242   
                                     p-value     0.713353  0.173890   
교육 서비스업(85)                          Beta       -0.011428  0.521202   
                                     p-value     0.885088  0.062089   
협회 및 단체, 수리 및 기타 개인 서비스 업(94~96)     Beta       -0.023347 -0.030388   
                                     p-value     0.667106  0.727171   
공공행정, 국방 및 사회보장 행정(84)               Beta       -0.052984 -0.102361   
                                     p-value     0.486043  0.199704   
출판, 영상, 방송통 신 및 정보서비스업 (58 ~ 63)     Beta        0.105814 -0.011913   
                                     p-value     0.100465  0.860735   
금융 및 보험업 (64 ~ 66)                   Beta        0.094475 -0.067515   
                                     p-value     0.487179  0.372247   
농업, 임업 및 어업 (01 ~ 03)                Beta        0.100659  0.010827   
                                     p-value     0.153268  0.937090   
부동산업 및 임대업 (68 ~ 69)                 Beta        0.005122 -0.120380   
                                     p-value     0.942516  0.078677   
숙박 및 음식점업 (55 ~ 56)                  Beta        0.105622  0.023670   
                                     p-value     0.115197  0.877032   
하수 · 폐기물 처리, 원료재생 및 환경복 원업 (37 ~ 39) Beta        0.018505 -0.220199   
                                     p-value     0.728561  0.201168   
전문, 과학 및 기술 서비스업 (70 ~ 73)           Beta        0.124800 -0.188472   
                                     p-value     0.235845  0.289708   
보건업 및 사회복지 서비스업(86~87)               Beta        0.051778 -0.044133   
                                     p-value     0.767128  0.585998   

                                              university  
제조업 (10 ~ 33)                        Beta      -0.049517  
                                     p-value    0.243791  
도매 및 소매업 (45~47)                     Beta       0.040709  
                                     p-value    0.408767  
건설업 (41 ~ 42)                        Beta       0.070167  
                                     p-value    0.292259  
운수업(49~52)                           Beta       0.158836  
                                     p-value    0.036222  
교육 서비스업(85)                          Beta       0.030587  
                                     p-value    0.282285  
협회 및 단체, 수리 및 기타 개인 서비스 업(94~96)     Beta       0.005717  
                                     p-value    0.943241  
공공행정, 국방 및 사회보장 행정(84)               Beta       0.042506  
                                     p-value    0.426225  
출판, 영상, 방송통 신 및 정보서비스업 (58 ~ 63)     Beta       0.013712  
                                     p-value    0.810491  
금융 및 보험업 (64 ~ 66)                   Beta      -0.013393  
                                     p-value    0.692681  
농업, 임업 및 어업 (01 ~ 03)                Beta       0.046098  
                                     p-value    0.731473  
부동산업 및 임대업 (68 ~ 69)                 Beta       0.045558  
                                     p-value    0.849233  
숙박 및 음식점업 (55 ~ 56)                  Beta       0.115993  
                                     p-value    0.442593  
하수 · 폐기물 처리, 원료재생 및 환경복 원업 (37 ~ 39) Beta       0.045685  
                                     p-value    0.641375  
전문, 과학 및 기술 서비스업 (70 ~ 73)           Beta       0.126790  
                                     p-value    0.231912  
보건업 및 사회복지 서비스업(86~87)               Beta       0.047315  
                                     p-value    0.640258

In [25]:
corr_dict = {}
for j in group_rid_dict.keys():
    corr_list = []
    for i in [0, 1, 2]:
        corr_list.append(np.corrcoef(temp_eta.loc[ksi[j].index, :].코스피, ksi[j][ksi[j].columns[i]])[0, 1])

    corr_dict[j] = pd.DataFrame(corr_list, index=ksi[j].columns, columns=["correlation coefficent"])

In [26]:
corr_df = pd.DataFrame.from_dict({(i,j): corr_dict[i][j]
                        for i in corr_dict.keys()
                        for j in corr_dict[i].columns}, orient="index")
corr_df.index = corr_df.index.levels[0]
corr_df

,high_school,college,university
건설업 (41 ~ 42),-0.054838,0.393805,-0.281959
"공공행정, 국방 및 사회보장 행정(84)",-0.026937,0.028403,0.169910
교육 서비스업(85),-0.449086,0.267177,0.226206
금융 및 보험업 (64 ~ 66),-0.159531,-0.328275,0.459501
"농업, 임업 및 어업 (01 ~ 03)",-0.046971,0.425055,0.239400
도매 및 소매업 (45~47),-0.149741,-0.107649,-0.026276
보건업 및 사회복지 서비스업(86~87),-0.176218,-0.279557,0.172870
부동산업 및 임대업 (68 ~ 69),0.360163,-0.066347,0.010570
숙박 및 음식점업 (55 ~ 56),0.155878,-0.226488,-0.116062
운수업(49~52),0.320428,0.045330,0.072702
